In [ ]:
import pandas as pd

In [4]:
datasets = pd.read_csv('/opt/ml/input/data/wiki_ner.csv')
datasets['context_ner'] = datasets['context_ner'].apply(eval)

In [5]:
datasets.head(2)

,text,corpus_source,url,domain,title,author,html,document_id,context_ner
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,NaN,나라 목록,NaN,NaN,0,"{'QUANTITY': [('206개', 20), ('두 부분', 75), ('첫 ..."
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,NaN,나라 목록,NaN,NaN,1,"{'DATE': [('1933년', 16)], 'ORGANIZATION': [('몬..."


In [6]:
type(datasets['context_ner'][0])

dict

In [7]:
datasets['context_ner'][:5]

0    {'QUANTITY': [('206개', 20), ('두 부분', 75), ('첫 ...
1    {'DATE': [('1933년', 16)], 'ORGANIZATION': [('몬...
2    {'CITY': [('서울특별시', 2), ('홍콩', 230), ('쾰른', 13...
3    {'ORGANIZATION': [('아오조라 문고', 0), ('아오조라 분고', ...
4    {'OCCUPATION': [('저자', 0), ('저자', 71), ('작가', ...
Name: context_ner, dtype: object

In [8]:
desired_keys = set(['QUANTITY', 'PERSON'])

def reshape_row(row, keys):
    new_rows = []
    for key, values in row['context_ner'].items():
        if key in keys:
            for value, start in values:
                new_row = {
                    'context': row['text'],
                    'document_id': row['document_id'],
                    'answer': value,
                    'answer_start': start,
                    'key': key   # Add the key
                }
                new_rows.append(new_row)
    return new_rows


In [9]:
desired_keys = set(['TERM', 'MATERIAL', 'PLANT', 'ANINAL', 'EVENT', 'TIME', 'CIVILIZATION', 'LOCATION', 'ORGANIZATION', 'ARTIFACTS', 'THEORY', 'STUDY_FIELD', 'QUANTITY', 'DATE', 'PERSON'])
new_rows = datasets.apply(lambda row: reshape_row(row, desired_keys), axis=1)
new_df = pd.concat([pd.DataFrame(v) for v in new_rows if v], ignore_index=True)


In [10]:
new_df['key'].value_counts()

key
PERSON          429708
CIVILIZATION    398200
DATE            335915
QUANTITY        310066
ORGANIZATION    290031
LOCATION        203549
TERM            149898
EVENT            61518
STUDY_FIELD      42655
MATERIAL         30834
THEORY           22440
TIME             14174
PLANT            10231
Name: count, dtype: int64

In [11]:
len(new_df)

2299219

In [12]:
new_df.loc[0]

context         이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...
document_id                                                     0
answer                                                       206개
answer_start                                                   20
key                                                      QUANTITY
Name: 0, dtype: object

In [13]:
def spacing(x):
    return x.replace(u'\xa0', u' ').replace('\u3000', ' ')
new_df['context'] =new_df['context'].apply(spacing)
new_df['answer'] =new_df['answer'].apply(spacing)


In [14]:
def preprocess2(x):
    if x['context'].find(x['answer']) > -1:
        return True
    else:
        return False

new_df['asdf'] =new_df.apply(preprocess2, axis=1)

In [15]:
new_df['asdf'].value_counts()

asdf
True     2299024
False        195
Name: count, dtype: int64

In [16]:
data = new_df[new_df['asdf'] == True]
data.drop('asdf', axis=1, inplace=True)

/tmp/ipykernel_4650/4280476082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('asdf', axis=1, inplace=True)


In [17]:
def preprocess(x):
    if len(x['context']) < 450:
        return x['context']
    else:
        idx = x['context'].index(x['answer'])
        return x['context'][max(0,idx-225):idx+225]
    
data['context'] =data.apply(preprocess, axis=1)

/tmp/ipykernel_4650/2196115205.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['context'] =data.apply(preprocess, axis=1)


In [18]:
data.loc[0]

context         이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...
document_id                                                     0
answer                                                       206개
answer_start                                                   20
key                                                      QUANTITY
Name: 0, dtype: object

In [20]:
len(data)

2299024

In [ ]:
data.to_csv('preprocessed_ner.csv', encoding='utf-8-sig')